In [5]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'


import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [6]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("myapp7") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/date=2022-05-25")
# Выведите схему датафрейма ниже
events.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- admins: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- channel_id: long (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- media: struct (nullable = true)
 |    |    |-- media_type: string (nullable = true)
 |    |    |-- src: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_channel_to: long (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- reaction_from: string (nullable = true)
 |    |-- reaction_type: string (nullable = true)
 |    |-- subscription_channel: long (nullable = true)
 |    |-- tags: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- user: string (nullable = true)
 |-- event_type: s

In [10]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window 
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("myapp8") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/date=2022-05-21")

window = Window().partitionBy('event.message_from').orderBy("event.message_from")

dfWithLag = events.withColumn("lag_7",F.lag("event.message_to", 7).over(window))

dfWithLag.select('event.message_from', 'lag_7') \
.filter(dfWithLag.lag_7.isNotNull()) \
.orderBy(F.col('event.message_from').desc()) \
.show(10, False)

+------------+------+
|message_from|lag_7 |
+------------+------+
|146005      |45711 |
|146005      |149488|
|146005      |149488|
|139087      |107225|
|139087      |149488|
|139087      |148775|
|139087      |56000 |
|139087      |70838 |
|139087      |85143 |
|139087      |112575|
+------------+------+
only showing top 10 rows

